In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df1 = pd.read_csv("records_final_series_ODI_bowlers.csv")

In [3]:
df1.head()

,Player,Country,Mat,Overs,Maidens,Runs,Wickets,4w,Best,Avg,S/R,E/R,Series_Code,H/A,Decade_Index
0,Illingworth R,England,1,8.0,1,50,3,0,Mar-50,16.67,21.33,4.69,193,Away,0
1,Stackpole K R,Australia,1,8.0,0,40,3,0,Mar-40,13.33,21.33,3.75,193,Home,0
2,Mallett A A,Australia,1,8.0,1,34,3,0,Mar-34,11.33,21.33,3.19,193,Home,0
3,McKenzie G D,Australia,1,7.4,0,22,2,0,Feb-22,11.00,30.00,2.20,193,Home,0
4,D'Oliveira B L,England,1,6.0,1,38,1,0,Jan-38,38.00,48.00,4.75,193,Away,0


In [4]:
#df1.sort_values(['Series_Code', 'Mat'], ascending=[True, False], inplace=True)

In [5]:
#df1

In [6]:
series = df1['Decade_Index'].values
number, series_unique = np.unique(series, return_inverse = True)

In [7]:
players = df1['Player'].values
player_names, player_indices = np.unique(players, return_index = True)
player_dict_temp_3 = {}
for i in df1.index:
    if df1["Player"][i] in player_dict_temp_3.keys():
        player_dict_temp_3[df1["Player"][i]] = player_dict_temp_3[df1["Player"][i]] + [i]
    else:
        player_dict_temp_3[df1["Player"][i]] = [i]

In [8]:
player_indices

array([  277,  3170,  1953, ...,  7363, 10371,  8658], dtype=int64)

In [9]:
series_codes = df1['Series_Code'].values
series_list, series_occurances, series_code = np.unique(series_codes, return_index=True, return_inverse = True)
#series_list

In [10]:
series_to_number = {'Australia':0, 'Pakistan':1, 'New Zealand':2, 'West Indies':3, 'England':4, 'India' :5, 'Sri Lanka':6, 'Bangladesh':7, 'Zimbabwe':8, 'South Africa':9, 'Other':10} 
#opposition_country =[]
series_country = []
j = 0
for i in series_occurances:
    home = 0
    away = 0
    j = i
    s_code = df1['Series_Code'][i]
    while 1 != 0:
        #print(j)
        if df1['H/A'][j] == 'Home' and home == 0:
            #print(df['Country'][j])
            country = df1['Country'][j]
            home = home + 1
        if df1['Series_Code'][j]!=s_code or home ==1:
            break    
        j = j + 1 
    if country in series_to_number.keys():
        series_country.append(series_to_number[country])
    else:
        series_country.append(10)

In [11]:
#series_country

In [12]:
#len(series_country)

In [13]:
series_list = list(series_list)

In [14]:
trps = np.zeros(656)

for i in df1.index:
    avg = df1["Avg"][i]
    if not np.isnan(avg):
        s = df1["Series_Code"][i]
        index = np.where(series_list == s)[0]
        #print(index)
        trps[index] += df1["Runs"][i]

In [15]:
trps


array([ 1687.,  1136.,  1090.,   235.,  1089.,   216.,   462.,   642.,
         763.,   832.,   470.,   246.,   302.,  5106.,   303.,   595.,
         988.,   350.,   246.,   942.,   812.,   577.,   546.,   614.,
         682.,   781.,  4144.,   253.,  4780.,   743.,   637.,   689.,
        6353.,   433.,   648.,  1008.,  6293.,   871.,   818.,   604.,
         904.,   526.,   836.,  1300.,   599.,  1336.,  6087.,  1077.,
         212.,   967.,  1155.,   900., 10677.,   532.,  1570.,   711.,
        6206.,  1227.,   884.,   453.,   727.,  1103.,  1273.,   479.,
         440.,  1259.,  1933.,  1429.,  7325.,  3864.,  1521.,   982.,
        1342.,   968.,  1368.,   734.,  1688.,  5976.,  1246.,   580.,
        1510.,   673.,   833.,  1306.,   680.,   793.,  2365.,  1544.,
        1835.,  1691.,  2680.,  5133.,  2234.,  1060.,  2094.,   986.,
       11543.,   989.,  2601.,  5037.,   315.,  1319.,  1843.,  1805.,
         314.,   202.,   901.,  1848.,  1862.,  1416.,   330.,  5456.,
      

In [16]:
arpspc = np.zeros(11)
tmpc = np.zeros(11)
for i in range(len(trps)):
    tmpc[series_country[i]] += df1['Mat'][series_occurances[i]]
    arpspc[series_country[i]] += trps[i]

In [17]:
arpc=arpspc/tmpc

In [18]:
avr_average = np.average(arpc)

In [19]:
weights_arpc = np.average(arpc)/arpc

In [20]:
weights_arpc

array([0.89970923, 1.09182535, 1.00160753, 1.05964208, 0.90486512,
       0.8976751 , 1.15058691, 1.13148148, 1.15612125, 0.93121552,
       0.89313509])

In [21]:
weighted_runs_arr = []
for i in df1.index:
    #print(series_code[i])
    w = weights_arpc[series_country[series_code[i]]]
    weighted_runs_arr.append(df1['Runs'][i]*w)
#weighted_runs_arr

In [22]:
wtrps = np.zeros(656)
twps = np.zeros(656)
tops = np.zeros(656)
tbps = np.zeros(656)
balls = np.zeros(len(df1.index))
for i in df1.index:
    avg = df1["Avg"][i]
    if not np.isnan(avg):
        wtrps[series_code[i]] += weighted_runs_arr[i]
        twps[series_code[i]] += df1["Wickets"][i]
        balls[i] = math.floor(df1["Overs"][i])*6 + (df1["Overs"][i]- math.floor(df1["Overs"][i]))*10
        tbps[series_code[i]] += balls[i]
        
    tops[series_code[i]] += df1["Overs"][i]

In [23]:
def bowl_calc(param):
    bowl1=np.zeros(len(df1.index))
    for i in df1.index:
        bowl1[i] = (wtrps[series_code[i]] - weighted_runs_arr[i])/(tops[series_code[i]]-df1["Overs"][i])
        e_r = df1[param][i]
        bowl1[i] = bowl1[i]/e_r
        if np.isnan(e_r) or  e_r == 0:
            bowl1[i] = 0
    return bowl1

In [24]:
def name(bowl, x):
    runs_metric = []
    player_dict_temp = {}


    for i in player_names:
        no_of_wickets=0
        spf_sum = 0
        no_series = 0
        no_balls = 0
        for j in player_dict_temp_3[i]:
            spf_sum += bowl[j]
            no_series += 1
            no_of_wickets += df1["Wickets"][j]
            #no_balls += balls[j]
        #if(no_balls > 4000):
        if(no_of_wickets > x):    
            #runs_metric_20.append(runs/no_of_matches)
            player_dict_temp[i] = spf_sum/no_series

    sum_values =sum(player_dict_temp.values())
    '''for k,v in s:
        sum_values += v
        #print(k,":",v)'''
    a = sum_values/len(player_dict_temp) 
    return player_dict_temp, a

In [25]:
def final_players(x):
    bowl = bowl_calc("Avg")
    bowl1 = bowl_calc("E/R")
    bowl2 = bowl_calc("S/R")
    player_dict_temp_25, a_25 = name(bowl, x)
    player_dict_temp_35, a_35 = name(bowl1, x)
    player_dict_temp_45, a_45 = name(bowl2, x)
    avg_temp = [a_25, a_35, a_45]
    w_temp = np.average(avg_temp)/avg_temp

    player_dict_temp_55 = {}

    for i in player_dict_temp_45.keys():
        player_dict_temp_55[i] = (w_temp[0]*player_dict_temp_25[i]+w_temp[1]*player_dict_temp_35[i]+w_temp[2]*player_dict_temp_45[i])/np.sum(w)

    s_55 = [(k,player_dict_temp_55[k]) for k in sorted(player_dict_temp_55, key=player_dict_temp_55.get, reverse = True)]
    return s_55

In [26]:
s_55 = final_players(50)
s_51 = final_players(150)

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  


In [34]:
def print_country_lists(start_decade):
    players_list = {}
    for country in series_to_number.keys():
        #print(country)
        count=0
        players_list[country] = list()
        for k,v in s_51:
            first = player_indices[np.where(player_names == k)[0][0]]
            #print(first)
            #print(first,"->",df[' Decade_Index '][first])
            if df1['Country'][first] == country and df1['Decade_Index'][first] >= start_decade:
                player = str(k)+':'+str(v)
                players_list[country] = players_list[country]+[player]
                count += 1
            if count == 5:
                break
        if count < 5:
            players_list[country] = list()
            count = 0
            for k,v in s_55:
                first = player_indices[np.where(player_names == k)[0][0]]
            #print(first)
            #print(first,"->",df[' Decade_Index '][first])
                if df1['Country'][first] == country and df1['Decade_Index'][first] >= start_decade:
                    player = str(k)+':'+str(v)
                    players_list[country] = players_list[country]+[player]
                    count += 1
                if count == 5:
                    break
    for k in players_list.keys():
        if k == "Other":
            break
        print(k)
        for i in players_list[k]:
            print(i)
        print("\n")

In [35]:
print_country_lists(0)

Australia
Johnson M G:1.3815468077793245
Bracken N W:1.3794290980038881
Lee B:1.356377547913768
McGrath G D:1.337629567850586
McDermott C J:1.2173478005761462


Pakistan
Saeed Ajmal:1.5655556707199536
Saqlain Mushtaq:1.4802732064558366
Shoaib Akhtar:1.4274659201409632
Wasim Akram:1.3675771697323083
Waqar Younis:1.3209364444330416


New Zealand
Mills K D:1.3883933419629175
Oram J D P:1.3032763981403555
Vettori D L:1.2250804004329532
Hadlee R J:1.1986584996542793
Southee T G:1.14293858987966


West Indies
Bravo D J J:1.2521211233528453
Ambrose C E L:1.2359503854457756
Walsh C A:1.1372213323535876
Marshall M D:1.1203622223461411
Gayle C H:1.0861712978976628


England
Woakes C R:1.353551178186593
Flintoff A:1.2966637496656928
Plunkett L E:1.2570953459948844
Finn S T:1.24332928897081
Rashid A U:1.2265038055502036


India
Agarkar A B:1.2700044528523404
Pathan I K:1.2497069197156498
Nehra A:1.246249856912438
Khan Z:1.2379587877682035
Srinath J:1.1570641559204227


Sri Lanka
Muralitharan M:1.4

In [36]:
print_country_lists(4)

Australia
Starc M A:1.6691568635562397
Bollinger D E:1.5566277316478672
Tait S W:1.460323202208161
Hazlewood J R:1.4521561289790192
Clark S R:1.4433709656274791


Pakistan
Saeed Ajmal:1.5655556707199536
Shoaib Akhtar:1.4274659201409632
Umar Gul:1.2497451078984265
Abdul Razzaq:1.2014840676751568
Shahid Afridi:1.1960967632869188


New Zealand
Bond S E:1.831382175339484
Henry M J:1.7474730778024836
Boult T A:1.6510473907690295
Santner M J:1.5768595300402708
Anderson C J:1.5027784008174307


West Indies
Narine S P:1.4883560990323024
Taylor J E:1.352452328421477
Rampaul R:1.3425899358142872
Bravo D J J:1.2682752666147041
Roach K A J:1.2582768758543845


England
Woakes C R:1.353551178186593
Flintoff A:1.2966637496656928
Plunkett L E:1.2570953459948844
Finn S T:1.24332928897081
Rashid A U:1.2265038055502036


India
Agarkar A B:1.2700044528523404
Pathan I K:1.2497069197156498
Nehra A:1.246249856912438
Khan Z:1.2379587877682035
Harbhajan Singh:1.1503194842939528


Sri Lanka
Mendis B A W:1.56363

In [31]:
def print_alltime_lists(start_decade):    
    for k,v in s_51:
        first = player_indices[np.where(player_names == k)[0][0]]
        #print(first,"->",df[' Decade_Index '][first])
        if df1['Decade_Index'][first] >= start_decade:
            print(k,":",v)

In [32]:
print_alltime_lists(0)

Saeed Ajmal : 1.5655556707199536
Saqlain Mushtaq : 1.4802732064558366
Steyn D W : 1.465844149274983
Muralitharan M : 1.4396337221346602
Shoaib Akhtar : 1.4274659201409632
Mills K D : 1.3883933419629175
Pollock S M : 1.3828233969217276
Johnson M G : 1.3815468077793245
Bracken N W : 1.3794290980038881
Wasim Akram : 1.3675771697323083
Malinga S L : 1.36499462730185
Morkel M : 1.3622556809414592
Lee B : 1.356377547913768
McGrath G D : 1.337629567850586
Shakib Al Hasan : 1.3366217603048678
Ntini M : 1.3324375263026331
Waqar Younis : 1.3209364444330416
Vaas W P U J C : 1.3200418819232809
Donald A A : 1.318213994801173
Oram J D P : 1.3032763981403555
Flintoff A : 1.2799214762293223
Fernando C R D : 1.2784618706619864
Agarkar A B : 1.2700044528523404
Imran Khan : 1.2572147063457957
Bravo D J J : 1.2521211233528453
Umar Gul : 1.2497451078984265
Pathan I K : 1.2497069197156498
Nehra A : 1.246249856912438
Kulasekara K M D N : 1.2433624647673926
Khan Z : 1.2379587877682035
Ambrose C E L : 1.235950

In [37]:
print_alltime_lists(4)

Saeed Ajmal : 1.5655556707199536
Steyn D W : 1.465844149274983
Shoaib Akhtar : 1.4274659201409632
Mills K D : 1.3883933419629175
Pollock S M : 1.3828233969217276
Johnson M G : 1.3815468077793245
Bracken N W : 1.3794290980038881
Malinga S L : 1.36499462730185
Morkel M : 1.3622556809414592
Lee B : 1.356377547913768
Shakib Al Hasan : 1.3366217603048678
Ntini M : 1.3324375263026331
Oram J D P : 1.3032763981403555
Flintoff A : 1.2799214762293223
Fernando C R D : 1.2784618706619864
Agarkar A B : 1.2700044528523404
Bravo D J J : 1.2521211233528453
Umar Gul : 1.2497451078984265
Pathan I K : 1.2497069197156498
Nehra A : 1.246249856912438
Kulasekara K M D N : 1.2433624647673926
Khan Z : 1.2379587877682035
Vettori D L : 1.2250804004329532
Anderson J M : 1.2035308666152478
Abdul Razzaq : 1.2014840676751568
Abdur Razzak : 1.1986789232768493
Shahid Afridi : 1.1960967632869188
Broad S C J : 1.1947096927021963
Mashrafe Mortaza : 1.1886624612749763
Harbhajan Singh : 1.1503194842939528
Southee T G : 1.1